# 라이브러리 임포트

In [43]:
import requests
import time
import json
import pandas as pd
import requests
from tqdm import tqdm

# 요청사항 설정

SF_0001	부산항	35.091	129.099   
SF_0002	부산항(신항)	35.023	128.808   
SF_0003	인천항	37.379	126.616   
SF_0004	평택·당진항	37.113	126.393   
SF_0005	군산항	35.974	126.586   
SF_0006	대산항	36.977	126.304   
SF_0007	목포항	34.751	126.309   
SF_0008	여수항	34.754	127.752   
SF_0009	해운대	35.159	129.16   
SF_0010	울산항	35.501	129.387   
SF_0011	포항항	36.051	129.378

In [182]:
인증키 = "jOlvCw5shKOqFfaIba0ow=="
관측소 = 'SF_0009'
날짜 = '20240925'

# 데이터 요청 함수

In [113]:
def download_file(key ,date, stn):
    url = f'http://www.khoa.go.kr/api/oceangrid/seafogReal/search.do?ServiceKey={key}&ObsCode={stn}&Date={date}&ResultType=json'
    # print(url)
    for attempt in range(5):  # 5회 시도
        try:
            response = requests.get(url, timeout=10)  # 10초 타임아웃
            response.raise_for_status()  # HTTP 오류 발생 시 예외 발생
            # JSON 데이터 파싱
            data = json.loads(response.content)  # json 데이터로 변환
            return data  # 파싱한 JSON 데이터 반환
        except requests.exceptions.Timeout:
            print(f"Attempt {attempt + 1} timed out, retrying...")
            time.sleep(1)  # 1초 대기 후 재시도
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data for {date}, station {stn}: {e}")
            return None
    
    print(f"Failed to fetch data for {date}, station {stn} after multiple attempts.")
    return None

# 데이터 요청

In [25]:
# json 타입
data = download_file(인증키, 날짜, 관측소)

In [31]:
# 일일 데이터 개수
len(data['result']['data'])

1440

# 피처 누락 확인
총 11개가 나와야 정상

In [199]:
# json 타입
data = download_file(인증키, '20240910', 관측소)
display(len(data['result']['data'][0]))
data['result']['data'][0]

8

{'obs_time': '2024-09-10 00:00',
 'obs_post_id': 'SF_0009',
 'obs_lat': '35.15909',
 'air-temp': '27.3',
 'obs_post_name': '해운대',
 'obs_lon': '129.16026',
 'vis': '20000',
 'air_pres': '1012.9'}

# 피처가 모두 있는 관측소 및 기간
0006 대산항: 2023 01 01   
0009 해운대: 2021 12 01 ~ 2023 07 20?



In [209]:
data = download_file(인증키, '20230720', 관측소)
display(len(data['result']['data'][0]))
data['result']['data'][0]

11

{'obs_time': '2023-07-20 00:00',
 'obs_post_id': 'SF_0009',
 'obs_lat': '35.15909',
 'water_temp': '17.5',
 'air-temp': '21.5',
 'obs_post_name': '해운대',
 'wind_dir': '동북동',
 'wind_speed': '2.4',
 'obs_lon': '129.16026',
 'vis': '1340',
 'air_pres': '1007.9'}

# json 파싱

In [184]:
# DataFrame에 저장할 컬럼 설정
columns = ['관측소ID', '관측소명', '관측시간', '경도', '위도', 
           '기온', '기압', '수온', '풍향', '시정', '풍속']

In [187]:
df = pd.DataFrame(columns=columns)

# 날짜 범위 설정
start_year = 2022
end_year = 2023

In [191]:
# 전체 진행 상태를 나타내는 tqdm
total_days = (end_year - start_year + 1) * 12 * 31 * 24 # 예상 날짜 수 (연도, 월, 일의 모든 경우)

# 날짜별로 데이터 수집
for year in range(start_year, end_year + 1):
    for month in range(1, 13):
        print(year, '년', month, '월')
        for day in range(1, 32):
            try:
                # 날짜를 yyyyMMdd 형식으로 변환
                date = f'{year:04}{month:02}{day:02}'
                
                # API 호출
                json_data = download_file(인증키, date, 관측소)
                
                # 데이터가 존재하는지 확인
                if json_data and 'result' in json_data and 'data' in json_data['result']:
                    # for entry in json_data['result']['data']:
                    for i in range(0, len(json_data['result']['data']), 30):
                        entry = json_data['result']['data'][i]
                        # 새로운 데이터를 임시 DataFrame으로 만들기
                        new_data = pd.DataFrame([{
                            '관측소ID': entry['obs_post_id'],
                            '관측소명': entry['obs_post_name'],
                            '관측시간': entry['obs_time'],
                            '경도': entry['obs_lon'],
                            '위도': entry['obs_lat'],
                            '기온': entry.get('air-temp', None),  # air_temp 값이 없을 경우 None 처리
                            '기압': entry.get('air_pres', None),
                            '수온': entry.get('water_temp', None),
                            '풍향': entry.get('wind_dir', None),
                            '시정': entry.get('vis', None),
                            '풍속': entry.get('wind_speed', None)
                        }])
                        
                        # 기존 DataFrame에 새로운 데이터 추가 (concat 사용)
                        df = pd.concat([df, new_data], ignore_index=True)
            except ValueError:
                print(f"Invalid date: {year}-{month}-{day}")
                continue  # 잘못된 날짜가 있으면 다음 루프로


Total: 29379day [45:27, 10.77day/s]

2022 년 1 월


2022 년 2 월


2022 년 3 월


2022 년 4 월


...

2023 년 9 월


2023 년 10 월


2023 년 11 월


2023 년 12 월


Total: 34659day [49:36, 11.64day/s]


2년치 데이터 약 35000개(2 * 365 * 48)


In [192]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64038 entries, 0 to 64037
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   관측소ID   64038 non-null  object
 1   관측소명    64038 non-null  object
 2   관측시간    64038 non-null  object
 3   경도      64038 non-null  object
 4   위도      64038 non-null  object
 5   기온      64009 non-null  object
 6   기압      64001 non-null  object
 7   수온      47067 non-null  object
 8   풍향      47498 non-null  object
 9   시정      63308 non-null  object
 10  풍속      47498 non-null  object
dtypes: object(11)
memory usage: 5.4+ MB


In [193]:
df.head()

,관측소ID,관측소명,관측시간,경도,위도,기온,기압,수온,풍향,시정,풍속
0,SF_0009,해운대,2022-01-01 00:00,129.16026,35.15909,0.1,1026.7,13.2,서북서,20000,4.6
1,SF_0009,해운대,2022-01-01 00:30,129.16026,35.15909,-0.2,1026.8,13.1,서북서,20000,5.3
2,SF_0009,해운대,2022-01-01 01:00,129.16026,35.15909,-0.3,1026.8,13.1,서북서,20000,5.8
3,SF_0009,해운대,2022-01-01 01:30,129.16026,35.15909,-0.7,1026.8,12.8,서북서,20000,5.2
4,SF_0009,해운대,2022-01-01 02:00,129.16026,35.15909,-0.5,1027,12.8,북서,20000,4.2


In [194]:
# 중복 처리된 데이터프레임을 저장
df.to_excel('해무 날씨(2022-2023).xlsx', index=False)

In [196]:
temp = df.dropna()
temp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 46697 entries, 0 to 56284
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   관측소ID   46697 non-null  object
 1   관측소명    46697 non-null  object
 2   관측시간    46697 non-null  object
 3   경도      46697 non-null  object
 4   위도      46697 non-null  object
 5   기온      46697 non-null  object
 6   기압      46697 non-null  object
 7   수온      46697 non-null  object
 8   풍향      46697 non-null  object
 9   시정      46697 non-null  object
 10  풍속      46697 non-null  object
dtypes: object(11)
memory usage: 4.3+ MB


In [210]:
df = pd.read_excel("해무 날씨(2022-2023).xlsx")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34659 entries, 0 to 34658
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   관측소ID   34659 non-null  object 
 1   관측소명    34659 non-null  object 
 2   관측시간    34659 non-null  object 
 3   경도      34659 non-null  float64
 4   위도      34659 non-null  float64
 5   기온      34642 non-null  float64
 6   기압      34638 non-null  float64
 7   수온      23530 non-null  float64
 8   풍향      23746 non-null  object 
 9   시정      34294 non-null  float64
 10  풍속      23746 non-null  float64
dtypes: float64(7), object(4)
memory usage: 2.9+ MB


In [211]:
df.head()

,관측소ID,관측소명,관측시간,경도,위도,기온,기압,수온,풍향,시정,풍속
0,SF_0009,해운대,2022-01-01 00:00,129.16026,35.15909,0.1,1026.7,13.2,서북서,20000.0,4.6
1,SF_0009,해운대,2022-01-01 00:30,129.16026,35.15909,-0.2,1026.8,13.1,서북서,20000.0,5.3
2,SF_0009,해운대,2022-01-01 01:00,129.16026,35.15909,-0.3,1026.8,13.1,서북서,20000.0,5.8
3,SF_0009,해운대,2022-01-01 01:30,129.16026,35.15909,-0.7,1026.8,12.8,서북서,20000.0,5.2
4,SF_0009,해운대,2022-01-01 02:00,129.16026,35.15909,-0.5,1027.0,12.8,북서,20000.0,4.2


In [212]:
temp = df.dropna()

In [213]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23345 entries, 0 to 26905
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   관측소ID   23345 non-null  object 
 1   관측소명    23345 non-null  object 
 2   관측시간    23345 non-null  object 
 3   경도      23345 non-null  float64
 4   위도      23345 non-null  float64
 5   기온      23345 non-null  float64
 6   기압      23345 non-null  float64
 7   수온      23345 non-null  float64
 8   풍향      23345 non-null  object 
 9   시정      23345 non-null  float64
 10  풍속      23345 non-null  float64
dtypes: float64(7), object(4)
memory usage: 2.1+ MB


In [214]:
temp.to_excel('해무 날씨 널값 제거(2022-2023).xlsx', index=False)

In [216]:
temp.to_csv('해무 날씨 널값 제거(2022-2023).csv', index=False)

# 라벨 추가
해무: 1   
해무X: 0

In [2]:
import pandas as pd
temp = pd.read_csv('해무 날씨 널값 제거(2022-2023).csv')

In [3]:
# 해무 기준 조절 가능
t = 1000

# 새로운 열 'label' 추가 (시정 <= 1000이면 1, 그렇지 않으면 0)
temp['label'] = temp['시정'].apply(lambda x: 1 if x <= t else 0)
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23345 entries, 0 to 23344
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   관측소ID   23345 non-null  object 
 1   관측소명    23345 non-null  object 
 2   관측시간    23345 non-null  object 
 3   경도      23345 non-null  float64
 4   위도      23345 non-null  float64
 5   기온      23345 non-null  float64
 6   기압      23345 non-null  float64
 7   수온      23345 non-null  float64
 8   풍향      23345 non-null  object 
 9   시정      23345 non-null  float64
 10  풍속      23345 non-null  float64
 11  label   23345 non-null  int64  
dtypes: float64(7), int64(1), object(4)
memory usage: 2.1+ MB


In [17]:
count_label_1 = temp['label'].sum()
print(f"label이 1인 개수: {count_label_1}")

label이 1인 개수: 83


In [18]:
temp.to_csv('해무 날씨 라벨링 추가(2022-2023).csv', index=False)
temp.to_excel('해무 날씨 라벨링 추가(2022-2023).xlsx', index=False)